In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
import mysql.connector
import ssl

# inisialisasi array untuk menampilkan hasil
id_detail = []
keterangan_lowongan = []
deskripsi_lowongan = []
stem_detail = []

# konektor ke database
db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='UnNgGrape'
        )
cursor = db.cursor()
add = 0

cursor.execute("SELECT id_lowongan, stem_detail, link_lowongan FROM lowongan WHERE link_lowongan LIKE 'https://karir.com%'")
raw_lowongan = cursor.fetchall()
for row in raw_lowongan:
    id_lowongan = row[0]

    if row[1] == "":
        link = row[2]
        r = Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
        gcontext = ssl.SSLContext()  # Only for gangstars
        response = urlopen(r, context=gcontext).read()
        soup = BeautifulSoup(response, "lxml")

        raw_detail = soup.find_all("div","b-opportunity-info__main")
        for p in raw_detail:

            # menggunakan try except karena ada beberapa estimasi keterangan yang tidak terlampir
            try:
                
                keterangan = p.find("div", "b-opportunity-info__main__left" ).get_text(separator=". ").replace("Fungsi Kerja.", "")
            except:
                keterangan = "-"
            
            # menggunakan try except karena ada beberapa deskripsi yang NoneType
            try:
                raw_deskripsi = p.find("div", "html-content b-opportunity-show__box" ).get_text(separator=". ")
                nltk_tokens = nltk.sent_tokenize(raw_deskripsi)
                deskripsi = ""

                for x in nltk_tokens:
                    if x == "Disclaimer:.":
                        break
                    deskripsi = deskripsi + " " + x
            except:
                deskripsi = "Eror: deskripsi Nonetype"

            raw_stem = keterangan + " " + deskripsi
            # define punctuation
            punctuations = '''!()-[]{};:=+`'",<>./|\?@#$%^&*_~'''

            # remove punctuation from the string
            no_punct = ""
            for char in raw_stem:
                if char not in punctuations:
                    no_punct = no_punct + char
                else:
                    no_punct = no_punct + " "

            # display the unpunctuated stringZZz
            raw_stem = no_punct.lower()
            
            # create stemmer
            # nltk
            ps = PorterStemmer()
            # sastrawi
            factory = StemmerFactory()
            stemmer = factory.create_stemmer()

            # stemming process
            # nltk
            words = word_tokenize(raw_stem)
            nltk_stemmer = ""
            for w in words:
                nltk_stemmer = nltk_stemmer + " " + ps.stem(w)
            # sastrawi
            output_stemmer = stemmer.stem(nltk_stemmer)

            cursor.execute("UPDATE lowongan SET stem_detail = '"+ output_stemmer +"' WHERE link_lowongan = '"+ link + "'")
            db.commit()
            
            add = add + 1
            
            id_detail.append(id_lowongan)
            keterangan_lowongan.append(keterangan)
            deskripsi_lowongan.append(deskripsi)
            stem_detail.append(output_stemmer)

print("Data has been added: " + str(add))
detail_dict ={'id':id_detail, 'keterangan':keterangan_lowongan, 'deskripsi':deskripsi_lowongan, 'stem':stem_detail}
df = pd.DataFrame(detail_dict, columns = ['id', 'keterangan', 'deskripsi', 'stem'])

df.sort_values('id', ascending=True)

Data has been added: 473


,id,keterangan,deskripsi,stem
0,20936,"Teknik, Sipil. Jenjang Karir. Pemula / Staf. ...",Persyaratan. - Pend. S1 Teknik Sipil. - Umur ...,teknik sipil jenjang karir mula staf alam tida...
1,20937,"Planning Production, Teknik, Elektro , Teknik...",Persyaratan. Kualifikasi\r\n. •\tMinimum D3/S...,plan product teknik elektro teknik umum teknik...
2,20938,"Teknik, Mesin. Jenjang Karir. Manajer - Depar...",Persyaratan. 1. Kemampuan analisa mesin produ...,teknik mesin jenjang karir manaj departemen al...
3,20939,"Teknik, Sipil. Jenjang Karir. Pemula / Staf. ...",Persyaratan. pria. usia maks 35 tahun. pend m...,teknik sipil jenjang karir mula staf alam tida...
4,20940,"Periklanan / Promosi, Penjualan dan Pemasaran...",Persyaratan. IPK minimal 3.00 Skala 4.00\r\n....,iklan promosi jual dan pasar jenjang karir mul...
...,...,...,...,...
468,21404,Manufaktur dan Produksi. Jenjang Karir. Pemul...,Persyaratan. Pria dan Wanita. Pendidikan Mini...,manufaktur dan produksi jenjang karir mula sta...
469,21405,Layanan Pelanggan. Jenjang Karir. Pemula / St...,Persyaratan. Pria / Wanita Min. SMK Jur Akunt...,layan langgan jenjang karir mula staf alam tid...
470,21406,Administrasi. Jenjang Karir. Pemula / Staf. P...,Persyaratan. Wanita. Pendidikan SMK (Jurusan ...,administrasi jenjang karir mula staf alam tida...
471,21407,Administrasi. Jenjang Karir. Pemula / Staf. P...,Persyaratan. - Good communication in English....,administrasi jenjang karir mula staf alam tida...
